Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
config = tf.ConfigProto()
config.gpu_options.allow_growth=True


First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (108000, 28, 28) (108000,)
Validation set (54000, 28, 28) (54000,)
Test set (18000, 28, 28) (18000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (108000, 784) (108000, 10)
Validation set (54000, 784) (54000, 10)
Test set (18000, 784) (18000, 10)


In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [100,784]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[100,784], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
	 [[Node: add_21/_25 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_411_add_21", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder', defined at:
  File "C:\Users\ricsi\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\ricsi\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\ricsi\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\ricsi\Anaconda3\envs\tensorflow

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
batch_size = 128
loss_coeff=0.005
learning_rate=0.1

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+
                        loss_coeff*(tf.nn.l2_loss(weights)+tf.nn.l2_loss(biases)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [39]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 35.087906
Minibatch accuracy: 10.2%
Validation accuracy: 11.2%
Minibatch loss at step 500: 10.327554
Minibatch accuracy: 73.4%
Validation accuracy: 71.8%
Minibatch loss at step 1000: 6.041248
Minibatch accuracy: 80.5%
Validation accuracy: 75.3%
Minibatch loss at step 1500: 3.712205
Minibatch accuracy: 78.9%
Validation accuracy: 77.2%
Minibatch loss at step 2000: 2.212940
Minibatch accuracy: 85.9%
Validation accuracy: 78.9%
Minibatch loss at step 2500: 1.549912
Minibatch accuracy: 85.9%
Validation accuracy: 80.2%
Minibatch loss at step 3000: 1.299761
Minibatch accuracy: 79.7%
Validation accuracy: 81.3%
Minibatch loss at step 3500: 1.141428
Minibatch accuracy: 75.8%
Validation accuracy: 81.8%
Minibatch loss at step 4000: 0.794714
Minibatch accuracy: 84.4%
Validation accuracy: 82.0%
Minibatch loss at step 4500: 0.624399
Minibatch accuracy: 84.4%
Validation accuracy: 82.2%
Minibatch loss at step 5000: 0.771325
Minibatch accuracy: 85.2%
Validation accur

In [7]:
batch_size = 128
neurons=1024
learning_rate=0.0001
loss_coeff=0.005
train=True



graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, neurons]))
  biases1 = tf.Variable(tf.zeros([neurons]))
  weights2 = tf.Variable(tf.truncated_normal([neurons, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  layer1 = tf.layers.dense(inputs=logits1,units=neurons,activation=tf.nn.relu,
                           use_bias=True,trainable=train,name="layer2")
  layer1_valid = tf.layers.dense(inputs=tf.matmul(tf_valid_dataset, weights1) + biases1,units=neurons,activation=tf.nn.relu,
                                 use_bias=True,trainable=train,name="layer2",reuse=True)
  layer1_test = tf.layers.dense(inputs=tf.matmul(tf_test_dataset, weights1) + biases1,units=neurons,activation=tf.nn.relu,
                                 use_bias=True,trainable=train,name="layer2",reuse=True)

  logits2 = tf.matmul(layer1, weights2) + biases2
  logits2_valid = tf.matmul(layer1_valid, weights2) + biases2
  logits2_test = tf.matmul(layer1_test, weights2) + biases2

  var_kernel = [v for v in tf.global_variables() if v.name == "layer2/kernel/read:0"]
  var_bias = [v for v in tf.global_variables() if v.name == "layer2/bias/read:0"]

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2)+
                       loss_coeff*(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2)+tf.nn.l2_loss(biases1)+
                                  tf.nn.l2_loss(biases2)+tf.nn.l2_loss(var_kernel)+tf.nn.l2_loss(var_bias)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(logits2_valid)
  test_prediction = tf.nn.softmax(logits2_test)
  #x=tf.trainable_variables()[4]


In [83]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1958.339844
Minibatch accuracy: 10.2%
Validation accuracy: 19.3%
Minibatch loss at step 500: 1601.791504
Minibatch accuracy: 72.7%
Validation accuracy: 78.9%
Minibatch loss at step 1000: 1590.234375
Minibatch accuracy: 85.2%
Validation accuracy: 80.8%
Minibatch loss at step 1500: 1590.325928
Minibatch accuracy: 85.9%
Validation accuracy: 81.3%
Minibatch loss at step 2000: 1580.421875
Minibatch accuracy: 89.1%
Validation accuracy: 81.6%
Minibatch loss at step 2500: 1577.648438
Minibatch accuracy: 90.6%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 1576.714600
Minibatch accuracy: 89.1%
Validation accuracy: 82.2%
Minibatch loss at step 3500: 1574.391602
Minibatch accuracy: 85.9%
Validation accuracy: 82.9%
Minibatch loss at step 4000: 1573.563232
Minibatch accuracy: 89.8%
Validation accuracy: 83.5%
Minibatch loss at step 4500: 1574.855835
Minibatch accuracy: 89.1%
Validation accuracy: 83.4%
Minibatch loss at step 5000: 1571.588623
Minibatch a

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [8]:
batch_size = 10
neurons=1024
learning_rate=0.0001
loss_coeff=0.0
train=True



graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, neurons]))
  biases1 = tf.Variable(tf.zeros([neurons]))
  weights2 = tf.Variable(tf.truncated_normal([neurons, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  layer1 = tf.layers.dense(inputs=logits1,units=neurons,activation=tf.nn.relu,
                           use_bias=True,trainable=train,name="layer2")
  layer1_valid = tf.layers.dense(inputs=tf.matmul(tf_valid_dataset, weights1) + biases1,units=neurons,activation=tf.nn.relu,
                                 use_bias=True,trainable=train,name="layer2",reuse=True)
  layer1_test = tf.layers.dense(inputs=tf.matmul(tf_test_dataset, weights1) + biases1,units=neurons,activation=tf.nn.relu,
                                 use_bias=True,trainable=train,name="layer2",reuse=True)

  logits2 = tf.matmul(layer1, weights2) + biases2
  logits2_valid = tf.matmul(layer1_valid, weights2) + biases2
  logits2_test = tf.matmul(layer1_test, weights2) + biases2

  var_kernel = [v for v in tf.global_variables() if v.name == "layer2/kernel/read:0"]
  var_bias = [v for v in tf.global_variables() if v.name == "layer2/bias/read:0"]

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2)+
                       loss_coeff*(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2)+tf.nn.l2_loss(biases1)+
                                  tf.nn.l2_loss(biases2)+tf.nn.l2_loss(var_kernel)+tf.nn.l2_loss(var_bias)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(logits2_valid)
  test_prediction = tf.nn.softmax(logits2_test)
  #x=tf.trainable_variables()[4]

In [89]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 82.050217
Minibatch accuracy: 50.0%
Validation accuracy: 13.4%
Minibatch loss at step 500: 573.706970
Minibatch accuracy: 50.0%
Validation accuracy: 71.2%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.0%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.9%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.0%
Minibatch loss at step 3000: 775.967590
Minibatch accuracy: 50.0%
Validation accuracy: 74.7%
Minibatch loss at step 3500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 4000: 207.410889
Minibatch accuracy: 0.0%
Validation accuracy: 77.3%
Minibatch loss at step 4500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 5000: 80.109032
Minibatch accuracy: 50.0%
Valid

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [39]:
batch_size = 200
hidden_nodes = 1024
no_layers = 5

learning_coeff=0.5
decay_steps=1000
decay_rate=0.9
loss_coeff=0.001
train=True
dropout=0.4

#84% at 30k with 0.000004
#84% at 16k with 0.000006
#84% at 10k with 0.00001
#85% at 13k with 0.00002
#85% at 7k  with 0.00003
#86% at 31k with 0.00001 loss coeff

#85% at 4k  with 0.00006

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  initial_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=np.sqrt(2.0 / image_size**2)))
  initial_biases = tf.Variable(tf.zeros([hidden_nodes]))
  initial_train_layer = lambda data: tf.nn.dropout(tf.nn.relu(tf.matmul(data, initial_weights) + initial_biases), dropout)
  initial_eval_layer = lambda data:tf.nn.relu(tf.matmul(data, initial_weights) + initial_biases)
    
  hidden_weights = [tf.Variable( tf.truncated_normal([hidden_nodes, hidden_nodes], stddev=np.sqrt(2.0 / hidden_nodes)))
                    for i in range(no_layers)]
  hidden_biases = [tf.Variable(tf.zeros([hidden_nodes])) for i in range(no_layers)]

    
  # Training computation.
  # layers
  hidden_train_layers = [lambda data: tf.nn.dropout(tf.nn.relu(
                         tf.matmul(data, hidden_weights[i]) + hidden_biases[i]), dropout)
                         for i in range(no_layers)]

  hidden_eval_layers = [lambda data: tf.nn.relu(
                        tf.matmul(data, hidden_weights[i]) + hidden_biases[i])
                        for i in range(no_layers)]

  final_weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels], stddev=np.sqrt(2.0 / hidden_nodes)))
  final_biases = tf.Variable(tf.zeros([num_labels]))
  final_layer = lambda data: tf.matmul(data, final_weights) + final_biases
      
    
  apply_layers = lambda layers, data:data if not layers else apply_layers(layers[1:], layers[0](data))

  train_layers = [initial_train_layer] + hidden_train_layers + [final_layer]
  train_stack = lambda data: apply_layers(train_layers, data)

  eval_layers = [initial_eval_layer] + hidden_eval_layers + [final_layer]
  eval_stack = lambda data: apply_layers(eval_layers, data)
  

  # final layer
  logits = train_stack(tf_train_dataset)
  l2_loss = loss_coeff*sum(map(tf.nn.l2_loss, [
    initial_weights, initial_biases, final_weights, final_biases] +
    hidden_weights + hidden_biases))
                   
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) + l2_loss
      
  # learning decay
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(learning_coeff, global_step, decay_steps,decay_rate)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(eval_stack(tf_valid_dataset))
  test_prediction = tf.nn.softmax(eval_stack(tf_test_dataset))
  #x=tf.trainable_variables()[4]

In [40]:
num_steps = 200001

with tf.Session(graph=graph,config=config) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    global_step=step
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 12.110424
Minibatch accuracy: 6.5%
Validation accuracy: 9.2%
Minibatch loss at step 500: 5.875794
Minibatch accuracy: 64.5%
Validation accuracy: 66.3%
Minibatch loss at step 1000: 3.875446
Minibatch accuracy: 81.5%
Validation accuracy: 82.8%
Minibatch loss at step 1500: 2.765948
Minibatch accuracy: 81.0%
Validation accuracy: 83.9%
Minibatch loss at step 2000: 2.001717
Minibatch accuracy: 83.5%
Validation accuracy: 84.4%
Minibatch loss at step 2500: 1.674191
Minibatch accuracy: 83.0%
Validation accuracy: 85.2%
Minibatch loss at step 3000: 1.354431
Minibatch accuracy: 83.5%
Validation accuracy: 85.7%
Minibatch loss at step 3500: 1.266570
Minibatch accuracy: 81.0%
Validation accuracy: 85.6%
Minibatch loss at step 4000: 0.996654
Minibatch accuracy: 87.5%
Validation accuracy: 86.5%
Minibatch loss at step 4500: 0.915852
Minibatch accuracy: 85.5%
Validation accuracy: 86.9%
Minibatch loss at step 5000: 0.674176
Minibatch accuracy: 90.0%
Validation accuracy

---
Recski Gábor kód!
---------


In [14]:
batch_size = 128
hidden_nodes = 1024

no_layers = 5

beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  
  # Variables.
  
  
  initial_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=np.sqrt(2.0 / image_size**2)))
  initial_biases = tf.Variable(tf.zeros([hidden_nodes]))
  initial_train_layer = lambda data: tf.nn.dropout(    
    tf.nn.relu(tf.matmul(data, initial_weights) + initial_biases), 0.5)
  initial_eval_layer = (lambda data:
                        tf.nn.relu(tf.matmul(data, initial_weights) + initial_biases))


  hidden_weights = [tf.Variable(
      tf.truncated_normal([hidden_nodes, hidden_nodes], stddev=np.sqrt(2.0 / hidden_nodes)))
                    for i in range(no_layers)]
  
  # hidden_weights = [
  #      tf.get_variable("W", shape=[hidden_nodes, hidden_nodes],
  #         initializer=tf.contrib.layers.xavier_initializer())for i in range(no_layers)]
  

  hidden_biases = [tf.Variable(tf.zeros([hidden_nodes])) for i in range(no_layers)]

  hidden_train_layers = [lambda data: tf.nn.dropout(tf.nn.relu(
                         tf.matmul(data, hidden_weights[i]) + hidden_biases[i]), 0.5)
                         for i in range(no_layers)]

  hidden_eval_layers = [lambda data: tf.nn.relu(
                        tf.matmul(data, hidden_weights[i]) + hidden_biases[i])
                        for i in range(no_layers)]

  final_weights = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels], stddev=np.sqrt(2.0 / hidden_nodes)))
  final_biases = tf.Variable(tf.zeros([num_labels]))
  final_layer = lambda data: tf.matmul(data, final_weights) + final_biases
      
    
  apply_layers = (lambda layers, data:
                  data if not layers else apply_layers(layers[1:], layers[0](data)))

  train_layers = [initial_train_layer] + hidden_train_layers + [final_layer]
  train_stack = lambda data: apply_layers(train_layers, data)

  eval_layers = [initial_eval_layer] + hidden_eval_layers + [final_layer]
  eval_stack = lambda data: apply_layers(eval_layers, data)

  # Training computation.
  logits = train_stack(tf_train_dataset)
  l2_loss = beta*sum(map(tf.nn.l2_loss, [
    initial_weights, initial_biases, final_weights, final_biases] +
    hidden_weights + hidden_biases))
                   
  loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) + l2_loss
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  decay_steps = 500
  decay_rate = 0.9
  learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(eval_stack(tf_valid_dataset))
  test_prediction = tf.nn.softmax(eval_stack(tf_test_dataset))

In [15]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 8.235756
Minibatch accuracy: 7.8%
Validation accuracy: 10.0%
Minibatch loss at step 500: 4.112888
Minibatch accuracy: 76.6%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 2.868176
Minibatch accuracy: 82.8%
Validation accuracy: 84.1%
Minibatch loss at step 1500: 2.110154
Minibatch accuracy: 85.2%
Validation accuracy: 84.9%
Minibatch loss at step 2000: 1.562528
Minibatch accuracy: 91.4%
Validation accuracy: 85.7%
Minibatch loss at step 2500: 1.382104
Minibatch accuracy: 86.7%
Validation accuracy: 86.3%
Minibatch loss at step 3000: 1.212774
Minibatch accuracy: 85.9%
Validation accuracy: 86.4%
Minibatch loss at step 3500: 1.256570
Minibatch accuracy: 81.2%
Validation accuracy: 87.1%
Minibatch loss at step 4000: 1.014832
Minibatch accuracy: 85.9%
Validation accuracy: 87.1%
Minibatch loss at step 4500: 0.877625
Minibatch accuracy: 84.4%
Validation accuracy: 87.6%
Minibatch